# SELENIUM
## RPA - AUTOMACAO DE CLIQUE EM LINK PÚBLICO
### <italic>Fonte de dados: tse.jus.br</italic>

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [33]:
from selenium.webdriver.chrome.options import Options
import os
import sys
import os.path

### Configurando o navegador para download:

In [32]:
def download_sem_cabecalho(browser, download_dir):
    browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    params = {'cmd':'Page.setDownloadBehavior', 'params':{'behavior':'allow','downloadPath': download_dir}}
    browser.execute("send_command", params)

### Selenium Webdriver

In [53]:
def download(url, seletorHtml):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": "c:\TESTE",
            "download.prompt_for_download":  True,
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            # caso o navegador detecte o arquivo como malicioso, ele mantem o download
            "safebrowsing.enabled": False 
    })
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-software-rasterizer')

    driver = webdriver.Chrome(executable_path="c:/bigdata/chromedriver.exe", options = chrome_options)
    download_dir = "c:\TESTE"
    download_sem_cabecalho(driver, download_dir)
    driver.get(url)
 
    search_input = driver.find_element_by_xpath(seletorHtml)

    if search_input:    
        try:
            search_input.click()            
            print('Elemento capturado >>>>> ', search_input.text)
            #print('downloading...')
            time.sleep(3)
                     
        except:
            print('erro')
    
    return search_input

#

### BeautifulSoup para criar lista com anos: 

In [ ]:
url = 'https://www.tse.jus.br/hotsites/pesquisas-eleitorais/candidatos_anos/2016.html'
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')
soup

In [ ]:
anos = []
for i in soup.findAll('li', class_='span-4'):
    ano = i.find('a').getText()
    if int(ano) >= 2010:
        anos.append(ano)

### Slice de url's do site

In [ ]:
url_download = 'https://www.tse.jus.br/hotsites/pesquisas-eleitorais/candidatos_anos/'
url_zip = 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_'
Xpath = lambda x : '//*[@id="conteudo"]/div[4]/p[1]/a' if x is not None else '//*[@id="conteudo"]/div[4]/p[2]/a'

### Funções de calculo de tamanho do arquivo e varredura

In [ ]:
def varredura(x):
    url = x
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    return Xpath(soup.find('p').find('a'))

def calcTamArquivo(nomeArquivo, ano):
    urlZip = '{}{}.zip'.format(nomeArquivo, ano)
    x = urlopen(urlZip)
    tam = x.info()['Content-length']
    return tam

def nomeArqLocal(nomeArquivo, ano):
    nome_do_arquivo = '{}{}.zip'.format(nomeArquivo.split('/')[-1], ano)
    caminho_arquivo_local = 'C:/TESTE/' + nome_do_arquivo
    return caminho_arquivo_local

### Loop que dispara o download automático

In [ ]:
for ano in anos:
    url = '{}{}.html'.format(url_download, ano)    
    tam_arquivo_rem = calcTamArquivo(url_zip, ano)
    tam_arquivo_rem = int(tam_arquivo_rem) / 1024
    print('Tamanho do arquivo remoto: {} MBytes'.format(str(tam_arquivo_rem)))
    download(url, varredura(url))    
    # aguardar final de um download e começar outro
    while not os.path.exists(nomeArqLocal(url_zip, ano)):        
        print('baixando...')
        time.sleep(5)
    print('Arquivo baixado com sucesso!')
 

### EOF